In [1]:
# Install required packages if not already installed
!pip install openai pandas numpy matplotlib # Uncomment this line if running in a new environment

# Individual Personality-Based LLM Demo

Test how an LLM with your personality makes decisions compared to you.








This demo is based on the study but is simplified for this class:
- Huang, M., Zhang, X., Soto, C., & Evans, J. (2025). *Designing AI-agents with personalities: A psychometric approach*. arXiv. https://arxiv.org/abs/2410.19238

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
import os
import re

## Step 1: Configure OpenAI API

In [3]:
# Enter your OpenAI API key (DO NOT SHARE THIS KEY)
OPENAI_API_KEY = ""


# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

## Step 2: Enter Your Information

Copy your data from the Google Drive CSV file.

In [4]:
# Your Participant ID
MY_ID = "Blue-Pancakes-7"  # Replace with your ID

# Your Personality String (copy from CSV)
MY_PERSONALITY = ""

## If you didn't fill it out, you can use this example:
# MY_PERSONALITY = """I am quite outgoing, sociable. I am very assertive. I am usually full of energy. I am fairly systematic, usually keep things in order. I am fairly efficient, get things done fairly quickly. I am very dependable, steady. I am sometimes tense. I rarely feel sad. I am not particularly moody, rarely have up and down mood swings."""

# Your actual survey responses (1-10 scale)
MY_RESPONSES = {
    "Exam_Cheating": 0,
    "Honest_Feedback": 0,
    "Investment": 0,
    "Entrepreneurial_Venture": 0
}

print(f"Participant: {MY_ID}")
print(f"\nYour responses: {MY_RESPONSES}")

Participant: Blue-Pancakes-7

Your responses: {'Exam_Cheating': 0, 'Honest_Feedback': 0, 'Investment': 0, 'Entrepreneurial_Venture': 0}


## Step 3: Test LLM with Your Personality

In [5]:
def test_llm(personality_string):
    prompt = f"""You are participating in a psychological study. You have been assigned the following personality profile:

{personality_string}

Based on this personality profile, please respond to the following scenarios. For each scenario, provide a rating from 1 to 10 that reflects how someone with your assigned personality would likely act.

### Scenario 1: Exam Cheating
You are a student in a community college. During the final exams, you notice your best friend, who has been under a lot of personal stress lately, subtly looking at cheat notes. This exam is crucial for their scholarship.

Scale: 1 = Being completely sympathetic and helping your friend out, 10 = Strictly adhering to ethical standards by reporting the cheating

### Scenario 2: Honest Feedback
A colleague and friend is up for a performance review that could affect their career. They've been underperforming and now seek your honest feedback.

Scale: 1 = Softening feedback to protect their feelings, 10 = Giving brutally honest feedback

### Scenario 3: Investment
You have money to invest. Choose between a high-risk stock and a safe government bond.

Scale: 1 = Choosing the high-risk stock, 10 = Choosing the safe government bond

### Scenario 4: Entrepreneurial Venture
You've developed a business idea that requires quitting your job and investing your savings.

Scale: 1 = Pursuing the entrepreneurial venture, 10 = Keeping your current job security

Please respond with ONLY a JSON object in this format:
{{
    "Exam_Cheating": [number 1-10],
    "Honest_Feedback": [number 1-10],
    "Investment": [number 1-10],
    "Entrepreneurial_Venture": [number 1-10]
}}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {"role": "system", "content": "Respond based on the assigned personality. JSON only."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=200
        )

        text = response.choices[0].message.content
        json_match = re.search(r'\{[^{}]*\}', text)

        if json_match:
            return json.loads(json_match.group())
        return json.loads(text)

    except Exception as e:
        print(f"Error: {e}")
        return None

# Test the LLM
print("Testing LLM...")
llm_responses = test_llm(MY_PERSONALITY)

if llm_responses:
    print("\nLLM Responses:")
    for scenario, score in llm_responses.items():
        print(f"  {scenario}: {score}")
else:
    print("Failed to get response")

Testing LLM...

LLM Responses:
  Exam_Cheating: 8
  Honest_Feedback: 7
  Investment: 3
  Entrepreneurial_Venture: 4


## Step 4: Compare Results

In [6]:
if llm_responses:
    # Calculate differences
    print("\nComparison:")
    print("-" * 50)
    print(f"{'Scenario':<25} {'You':>5} {'LLM':>5} {'Diff':>6}")
    print("-" * 50)

    diffs = []
    for scenario in MY_RESPONSES:
        human = MY_RESPONSES[scenario]
        llm = llm_responses.get(scenario, 0)
        diff = abs(human - llm)
        diffs.append(diff)
        print(f"{scenario:<25} {human:>5} {llm:>5} {diff:>6}")

    # Calculate similarity
    avg_diff = np.mean(diffs)
    similarity = max(0, 100 - (avg_diff * 10))

    print("-" * 50)
    print(f"\nSimilarity: {similarity:.1f}%")
    print(f"Average difference: {avg_diff:.1f} points")


Comparison:
--------------------------------------------------
Scenario                    You   LLM   Diff
--------------------------------------------------
Exam_Cheating                 0     8      8
Honest_Feedback               0     7      7
Investment                    0     3      3
Entrepreneurial_Venture       0     4      4
--------------------------------------------------

Similarity: 45.0%
Average difference: 5.5 points


## Step 5: Visualize

In [ ]:
if llm_responses:
    scenarios = list(MY_RESPONSES.keys())
    human_scores = list(MY_RESPONSES.values())
    llm_scores = [llm_responses.get(s, 0) for s in scenarios]

    # Create plot
    fig, ax = plt.subplots(figsize=(10, 6))

    x = np.arange(len(scenarios))
    width = 0.35

    bars1 = ax.bar(x - width/2, human_scores, width, label='You', color='#2E86AB')
    bars2 = ax.bar(x + width/2, llm_scores, width, label='LLM', color='#A23B72')

    ax.set_ylabel('Score')
    ax.set_title('Your Decisions vs LLM Decisions')
    ax.set_xticks(x)
    ax.set_xticklabels([s.replace('_', ' ') for s in scenarios], rotation=45, ha='right')
    ax.legend()
    ax.set_ylim(0, 11)
    ax.grid(True, alpha=0.3)

    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                   f'{height:.0f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

## Reflection

Consider:
- How similar were the LLM's decisions to yours?
- Which scenarios showed the biggest differences?
- Does the LLM behavior match your personality profile?

# Optional Extension:

## Part 2: Cultural Adaptation & Communication

In this section, you'll explore how personality relates to comfort in different cultural communication contexts.

## Cultural Communication Scenarios

Read the following two scenarios carefully. You'll be asked to rate your comfort level in each situation.

### Scenario 1 – High-Context Communication Culture

You've just been assigned by your company to relocate to a new country. In your new role, you quickly notice that much of the communication happens between the lines. People rarely state things directly; instead, meaning is conveyed through shared history, subtle cues, tone, and relationships. Informal gatherings after work or brief hallway chats often carry as much weight as formal meetings. To integrate, you need to pay close attention to non-verbal signals, read the atmosphere in the room, and invest time in building trust before colleagues feel comfortable being fully open. Success here depends as much on understanding the web of relationships as on your technical skills.

### Scenario 2 – Low-Context Communication Culture

You've just been assigned by your company to relocate to a new country. On your first day, you realize that conversations are refreshingly direct and explicit. Colleagues say exactly what they mean, and documents spell out expectations clearly, leaving little room for ambiguity. Meetings are structured, agendas are followed, and feedback tends to be candid, even when critical. You don't have to infer meaning from social context or long-standing relationships; instead, clarity and efficiency are valued. To succeed, you lean into being straightforward, asking precise questions, and stating your views clearly, which helps you gain credibility quickly.

## Step 6: Enter Your Cultural Adaptation Responses

Rate each statement from 1 (Strongly Disagree) to 7 (Strongly Agree) for both scenarios.

**For each scenario, rate:**
- "I feel comfortable adapting to everyday life in my new country."
- "I am confident in my ability to understand and adjust to local cultural norms."
- "I feel at ease interacting with people from the host culture in work and social settings."

In [9]:
# Your responses for HIGH-CONTEXT culture (Scenario 1)
# Rate from 1 (Strongly Disagree) to 7 (Strongly Agree)
MY_HIGH_CONTEXT_RESPONSES = {
    "comfort_everyday_life": 0,      # "I feel comfortable adapting to everyday life..."
    "confidence_adjust_norms": 0,    # "I am confident in my ability to understand and adjust..."
    "ease_interactions": 0           # "I feel at ease interacting with people..."
}

# Your responses for LOW-CONTEXT culture (Scenario 2)
# Rate from 1 (Strongly Disagree) to 7 (Strongly Agree)
MY_LOW_CONTEXT_RESPONSES = {
    "comfort_everyday_life": 0,      # "I feel comfortable adapting to everyday life..."
    "confidence_adjust_norms": 0,    # "I am confident in my ability to understand and adjust..."
    "ease_interactions": 0           # "I feel at ease interacting with people..."
}

print("✓ High-Context Culture Responses:", MY_HIGH_CONTEXT_RESPONSES)
print("✓ Low-Context Culture Responses:", MY_LOW_CONTEXT_RESPONSES)

✓ High-Context Culture Responses: {'comfort_everyday_life': 0, 'confidence_adjust_norms': 0, 'ease_interactions': 0}
✓ Low-Context Culture Responses: {'comfort_everyday_life': 0, 'confidence_adjust_norms': 0, 'ease_interactions': 0}


## Research Hypotheses

Now that you've provided your responses, let's consider what research predicts about personality and cultural adaptation:

**H1.** Extroverts will report higher levels of comfort in expatriate assignments than introverts.

**H2.** In high-context communication cultures, introverts will report higher levels of comfort than extroverts.

**H3.** In low-context communication cultures, extroverts will report higher levels of comfort than introverts.

Let's see how well an LLM with your personality matches your responses!

## Step 7: Test LLM on Cultural Scenarios

In [7]:
def test_cultural_adaptation(personality_string):
    prompt = f"""You are participating in a cross-cultural psychology study. You have been assigned the following personality profile:

{personality_string}

Based on this personality profile, please respond to two cultural scenarios. For each scenario, rate three statements on a scale from 1 to 7.

### Scenario 1 – High-Context Communication Culture
You've just been assigned by your company to relocate to a new country. In your new role, you quickly notice that much of the communication happens between the lines. People rarely state things directly; instead, meaning is conveyed through shared history, subtle cues, tone, and relationships. Informal gatherings after work or brief hallway chats often carry as much weight as formal meetings. To integrate, you need to pay close attention to non-verbal signals, read the atmosphere in the room, and invest time in building trust before colleagues feel comfortable being fully open. Success here depends as much on understanding the web of relationships as on your technical skills.

Rate these statements (1 = Strongly Disagree, 7 = Strongly Agree):
- \"I feel comfortable adapting to everyday life in my new country.\"
- \"I am confident in my ability to understand and adjust to local cultural norms.\"
- \"I feel at ease interacting with people from the host culture in work and social settings.\"

### Scenario 2 – Low-Context Communication Culture
You've just been assigned by your company to relocate to a new country. On your first day, you realize that conversations are refreshingly direct and explicit. Colleagues say exactly what they mean, and documents spell out expectations clearly, leaving little room for ambiguity. Meetings are structured, agendas are followed, and feedback tends to be candid, even when critical. You don't have to infer meaning from social context or long-standing relationships; instead, clarity and efficiency are valued. To succeed, you lean into being straightforward, asking precise questions, and stating your views clearly, which helps you gain credibility quickly.

Rate the same three statements (1 = Strongly Disagree, 7 = Strongly Agree).

Please respond with ONLY a JSON object in this format:
{{
    \"high_context\": {{
        \"comfort_everyday_life\": [number 1-7],
        \"confidence_adjust_norms\": [number 1-7],
        \"ease_interactions\": [number 1-7]
    }},
    \"low_context\": {{
        \"comfort_everyday_life\": [number 1-7],
        \"confidence_adjust_norms\": [number 1-7],
        \"ease_interactions\": [number 1-7]
    }}
}}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=[
                {"role": "system", "content": "Respond based on the assigned personality. JSON only."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=300
        )

        text = response.choices[0].message.content
        json_match = re.search(r'\\{[\\s\\S]*\\}', text)

        if json_match:
            return json.loads(json_match.group())
        return json.loads(text)

    except Exception as e:
        print(f"Error: {e}")
        return None

# Initialize OpenAI client (if not already done)
client = OpenAI(api_key=OPENAI_API_KEY)

# Test the LLM on cultural scenarios
print("Testing LLM on cultural adaptation scenarios...\n")
cultural_llm_responses = test_cultural_adaptation(MY_PERSONALITY)

if cultural_llm_responses:
    print("✓ LLM Responses Received\n")
    print("High-Context Culture:")
    for item, score in cultural_llm_responses['high_context'].items():
        print(f"  {item.replace('_', ' ').title()}: {score}")
    print("\nLow-Context Culture:")
    for item, score in cultural_llm_responses['low_context'].items():
        print(f"  {item.replace('_', ' ').title()}: {score}")
else:
    print("❌ Failed to get response from LLM")

Testing LLM on cultural adaptation scenarios...

✓ LLM Responses Received

High-Context Culture:
  Comfort Everyday Life: 4
  Confidence Adjust Norms: 3
  Ease Interactions: 4

Low-Context Culture:
  Comfort Everyday Life: 6
  Confidence Adjust Norms: 6
  Ease Interactions: 6


## Step 8: Compare Cultural Adaptation Results

In [10]:
if cultural_llm_responses:
    # Calculate average comfort scores
    def avg_score(responses):
        return np.mean(list(responses.values()))

    # Your scores
    your_high = avg_score(MY_HIGH_CONTEXT_RESPONSES)
    your_low = avg_score(MY_LOW_CONTEXT_RESPONSES)

    # LLM scores
    llm_high = avg_score(cultural_llm_responses['high_context'])
    llm_low = avg_score(cultural_llm_responses['low_context'])

    print("\n" + "="*60)
    print("CULTURAL ADAPTATION COMPARISON")
    print("="*60)

    print(f"\n{'Scenario':<30} {'You':>10} {'LLM':>10} {'Diff':>10}")
    print("-"*60)
    print(f"{'High-Context Culture':<30} {your_high:>10.2f} {llm_high:>10.2f} {abs(your_high - llm_high):>10.2f}")
    print(f"{'Low-Context Culture':<30} {your_low:>10.2f} {llm_low:>10.2f} {abs(your_low - llm_low):>10.2f}")
    print("-"*60)

    # Individual item comparison
    print("\n\nDETAILED ITEM COMPARISON")
    print("="*60)

    print("\nHigh-Context Culture:")
    print(f"{'Item':<35} {'You':>8} {'LLM':>8} {'Diff':>8}")
    print("-"*60)
    for item in MY_HIGH_CONTEXT_RESPONSES:
        you = MY_HIGH_CONTEXT_RESPONSES[item]
        llm = cultural_llm_responses['high_context'][item]
        print(f"{item.replace('_', ' ').title():<35} {you:>8} {llm:>8} {abs(you-llm):>8.1f}")

    print("\nLow-Context Culture:")
    print(f"{'Item':<35} {'You':>8} {'LLM':>8} {'Diff':>8}")
    print("-"*60)
    for item in MY_LOW_CONTEXT_RESPONSES:
        you = MY_LOW_CONTEXT_RESPONSES[item]
        llm = cultural_llm_responses['low_context'][item]
        print(f"{item.replace('_', ' ').title():<35} {you:>8} {llm:>8} {abs(you-llm):>8.1f}")


CULTURAL ADAPTATION COMPARISON

Scenario                              You        LLM       Diff
------------------------------------------------------------
High-Context Culture                 0.00       3.67       3.67
Low-Context Culture                  0.00       6.00       6.00
------------------------------------------------------------


DETAILED ITEM COMPARISON

High-Context Culture:
Item                                     You      LLM     Diff
------------------------------------------------------------
Comfort Everyday Life                      0        4      4.0
Confidence Adjust Norms                    0        3      3.0
Ease Interactions                          0        4      4.0

Low-Context Culture:
Item                                     You      LLM     Diff
------------------------------------------------------------
Comfort Everyday Life                      0        6      6.0
Confidence Adjust Norms                    0        6      6.0
Ease Interactions 

## Step 9: Visualize Cultural Adaptation Results

In [ ]:
if cultural_llm_responses:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Average scores comparison
    scenarios = ['High-Context\nCulture', 'Low-Context\nCulture']
    your_scores = [avg_score(MY_HIGH_CONTEXT_RESPONSES), avg_score(MY_LOW_CONTEXT_RESPONSES)]
    llm_scores = [avg_score(cultural_llm_responses['high_context']),
                  avg_score(cultural_llm_responses['low_context'])]

    x = np.arange(len(scenarios))
    width = 0.35

    bars1 = ax1.bar(x - width/2, your_scores, width, label='You', color='#2E86AB')
    bars2 = ax1.bar(x + width/2, llm_scores, width, label='LLM', color='#A23B72')

    ax1.set_ylabel('Average Comfort Score (1-7)', fontsize=11)
    ax1.set_title('Cultural Adaptation: Average Comfort Levels', fontsize=13, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(scenarios)
    ax1.legend()
    ax1.set_ylim(0, 8)
    ax1.grid(True, alpha=0.3, axis='y')

    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.2f}', ha='center', va='bottom', fontsize=10)

    # Detailed items comparison
    items = ['Comfort\nEveryday Life', 'Confidence\nAdjust Norms', 'Ease\nInteractions']
    item_keys = ['comfort_everyday_life', 'confidence_adjust_norms', 'ease_interactions']

    your_high_vals = [MY_HIGH_CONTEXT_RESPONSES[k] for k in item_keys]
    your_low_vals = [MY_LOW_CONTEXT_RESPONSES[k] for k in item_keys]
    llm_high_vals = [cultural_llm_responses['high_context'][k] for k in item_keys]
    llm_low_vals = [cultural_llm_responses['low_context'][k] for k in item_keys]

    x2 = np.arange(len(items))
    width2 = 0.2

    ax2.bar(x2 - 1.5*width2, your_high_vals, width2, label='You (High-Context)', color='#2E86AB')
    ax2.bar(x2 - 0.5*width2, llm_high_vals, width2, label='LLM (High-Context)', color='#A23B72')
    ax2.bar(x2 + 0.5*width2, your_low_vals, width2, label='You (Low-Context)', color='#5BA3C7')
    ax2.bar(x2 + 1.5*width2, llm_low_vals, width2, label='LLM (Low-Context)', color='#C274A8')

    ax2.set_ylabel('Comfort Score (1-7)', fontsize=11)
    ax2.set_title('Cultural Adaptation: Detailed Item Comparison', fontsize=13, fontweight='bold')
    ax2.set_xticks(x2)
    ax2.set_xticklabels(items)
    ax2.legend(fontsize=8, loc='upper left')
    ax2.set_ylim(0, 8)
    ax2.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

# Optional Activities - Test Your Own Theories

Use this space to design your own scenarios and test additional hypotheses about personality and decision-making.